### Tool to fetch internal data: getting specific review info

NOTE: Use SQL Warehouse to run this notebook as all cells are SQL

In [ ]:
%sql
select review_id, * from juan_dev.genai.amazon_reviews_mktpl where contains(product_name, "Soup") limit 10;

In [ ]:
%sql
select review_id, * from juan_dev.genai.amazon_reviews_mktpl where contains(product_name, "Bluetooth Earbuds ") limit 10;

In [ ]:
%sql
CREATE OR REPLACE FUNCTION juan_dev.genai.get_review(review_id_p STRING)
RETURNS TABLE(
  product_name STRING,
  product_rating DOUBLE,
  review_header STRING,
  review_text STRING)
COMMENT 'Returns a list of reviews for the given review ID (expect a UUID)'
LANGUAGE SQL
    RETURN
    SELECT product_name, product_rating, review_header, review_text 
    FROM juan_dev.genai.amazon_reviews_mktpl 
    WHERE review_id = review_id_p;

In [ ]:
%sql
SELECT * FROM juan_dev.genai.get_review("RZH2X8IQWV0I8");

- https://docs.databricks.com/aws/en/generative-ai/agent-framework/unstructured-retrieval-tools

NOTE: response can come back with non semantic results given the small data set size. 

In [0]:
-- ALready created vector search in prior step
SELECT PRODUCT_NAME, PRODUCT_RATING, RATING, REVIEW_ID, REVIEW_TEXT, BRAND, HELPFUL_COUNT FROM
  vector_search(
    -- Specify your Vector Search index name here
    index => 'juan_dev.genai.amazon_reviews_mktpl_vsidx',
    query => "looking to purchase Bluetooth Earbuds that have good frequency response in mid range",
    num_results => 4
  )

In [0]:
CREATE OR REPLACE FUNCTION juan_dev.genai.semantic_search_reviews (
  -- The agent uses this comment to determine how to generate the query string parameter.
  query STRING
  COMMENT 'The query string for searching amazon reviews.'
) RETURNS TABLE
-- The agent uses this comment to determine when to call this tool. It describes the types of documents and information contained within the index.
COMMENT 'Executes a search on amazon reviews most relevant to the input query.' 
RETURN
SELECT * FROM
  vector_search(
    -- Specify your Vector Search index name here
    index => 'juan_dev.genai.amazon_reviews_mktpl_vsidx',
    query => query,
    num_results => 3
  )

In [0]:
-- ********* NOTE - THIS ONLY WORKS ON SQL WAREHOUSE COMPUTE, Running on Cluster will error ******** ------
select * from juan_dev.genai.semantic_search_reviews("looking for reviews of high res 4k monitors")

In [0]:
select REVIEW_ID, PRODUCT_NAME, PRODUCT_RATING, RATING, REVIEW_TEXT, BRAND, HELPFUL_COUNT
from juan_dev.genai.semantic_search_reviews("looking to purchase Bluetooth Earbuds for small ears and prefer cheap brands");